In [3]:
import pandas as pd

In [4]:
df=pd.read_csv(r"C:\Users\fukai\Downloads\cfc_synthetic_cyber_policies.csv", parse_dates=['InceptionDate','ExpiryDate'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   PolicyNo            5000 non-null   object        
 1   InceptionDate       5000 non-null   datetime64[ns]
 2   ExpiryDate          5000 non-null   datetime64[ns]
 3   Region              5000 non-null   object        
 4   Industry            5000 non-null   object        
 5   RevenueBand         5000 non-null   object        
 6   Employees           5000 non-null   int64         
 7   CyberControlsScore  5000 non-null   float64       
 8   Underwriter         5000 non-null   object        
 9   Premium             5000 non-null   float64       
 10  ClaimsCount         5000 non-null   int64         
 11  ClaimsAmount        5000 non-null   float64       
 12  Year                5000 non-null   int64         
 13  Month               5000 non-null   int64       

In [6]:
df.describe()

,InceptionDate,ExpiryDate,Employees,CyberControlsScore,Premium,ClaimsCount,ClaimsAmount,Year,Month,LossRatio,Frequency,Severity
count,5000,5000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,5000.000000,5000.000000
mean,2024-06-30 12:02:35.520000,2025-06-30 12:02:35.520000,83.323400,64.529500,2767.364718,0.060400,2632.461688,2024.0,6.487800,4.872931,0.060400,2558.867283
min,2024-01-01 00:00:00,2024-12-31 00:00:00,1.000000,10.000000,182.450000,0.000000,0.000000,2024.0,1.000000,0.000000,0.000000,0.000000
25%,2024-04-02 00:00:00,2025-04-02 00:00:00,5.000000,54.300000,250.917500,0.000000,0.000000,2024.0,4.000000,0.000000,0.000000,0.000000
50%,2024-06-27 00:00:00,2025-06-27 00:00:00,17.000000,64.700000,847.025000,0.000000,0.000000,2024.0,6.000000,0.000000,0.000000,0.000000
75%,2024-09-30 00:00:00,2025-09-30 00:00:00,40.000000,75.100000,2757.287500,0.000000,0.000000,2024.0,9.000000,0.000000,0.000000,0.000000
max,2024-12-30 00:00:00,2025-12-30 00:00:00,1993.000000,100.000000,28358.770000,2.000000,242493.290000,2024.0,12.000000,982.032519,2.000000,242493.290000
std,NaN,NaN,263.052303,15.254052,5022.884013,0.244875,14136.202605,0.0,3.442415,36.481402,0.244875,13722.213089


In [7]:
total_premium = df['Premium'].sum()

In [8]:
total_claim = df['ClaimsAmount'].sum()

In [9]:
overall_loss_ratio = total_claim/total_premium

In [10]:
print(total_premium,total_claim,overall_loss_ratio)

13836823.59 13162308.44 0.9512521681285668


In [11]:
# Cohort example:average loss ratio by RevenueBand
df.groupby('RevenueBand').agg(
    Policies=('PolicyNo','count'),
    AvgLossRatio=('LossRatio','mean'),
    Freq=('ClaimsCount','mean'),
    AvgSeverity=('Severity','mean')
).sort_values('Policies',ascending=False)

,Policies,AvgLossRatio,Freq,AvgSeverity
RevenueBand,,,,
100k-500k,2045,3.177340,0.058191,2602.140993
<100k,1296,13.075086,0.063272,2856.207052
500k-2m,940,0.821658,0.060638,2190.178133
2m-10m,485,0.233526,0.049485,1888.284990
10m+,234,0.154111,0.085470,3404.820855


In [12]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
# small example: model frequency as function of controls and industry dummies
df['freq'] = df['ClaimsCount']
model = smf.glm('freq ~ CyberControlsScore + C(Industry)', data=df, family=sm.families.Poisson()).fit()
print(model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:                 5000
Model:                            GLM   Df Residuals:                     4993
Model Family:                 Poisson   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1129.5
Date:                Mon, 01 Sep 2025   Deviance:                       1666.2
Time:                        12:06:53   Pearson chi2:                 4.91e+03
No. Iterations:                     6   Pseudo R-squ. (CS):            0.01020
Covariance Type:            nonrobust                                         
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
